In [2]:
import torch
from PIL import Image
from torchvision.transforms import transforms

from unet.dataset import DeadwoodDataset

Image.MAX_IMAGE_PIXELS = 2000000000

In [3]:
transforms_image = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)
transforms_mask = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)
dataset = DeadwoodDataset(
    "data/images", "data/masks", transforms_image, transforms_mask
)

In [4]:
image, mask = dataset[0]